# Modeling and Simulation in Python

Bungee dunk example

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

### Bungee jumping

Suppose you want to set the world record for the highest "bungee dunk", [as shown in this video](https://www.youtube.com/watch?v=UBf7WC19lpw).  Since the record is 70 m, let's design a jump for 80 m.

We'll make the following modeling assumptions:

1. Initially the bungee cord hangs from a crane with the attachment point 80 m above a cup of tea.

2. Until the cord is fully extended, it applies no force to the jumper.  It turns out this might not be a good assumption; we will revisit it.

3. After the cord is fully extended, it obeys [Hooke's Law](https://en.wikipedia.org/wiki/Hooke%27s_law); that is, it applies a force to the jumper proportional to the extension of the cord beyond its resting length.

4. The jumper is subject to drag force proportional to the square of their velocity, in the opposite of their direction of motion.

Our objective is to choose the length of the cord, `L`, and its spring constant, `k`, so that the jumper falls all the way to the tea cup, but no farther! 

First I'll create a `Param` object to contain the quantities we'll need:

1. Let's assume that the jumper's mass is 75 kg.

2. With a terminal velocity of 60 m/s.

3. The length of the bungee cord is `L = 40 m`.

4. The spring constant of the cord is `k = 20 N / m` when the cord is stretched, and 0 when it's compressed.


In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
N = UNITS.newton

<Unit('newton')>

In [3]:
params = Params(y_attach = 80 * m,
                v_init = 0 * m / s,
                g = 9.8 * m/s**2,
                mass = 75 * kg,
                area = 1 * m**2,
                rho = 1.2 * kg/m**3,
                v_term = 60 * m / s,
                L = 25 * m,
                k = 40 * N / m)

,values
y_attach,80 meter
v_init,0.0 meter / second
g,9.8 meter / second ** 2
mass,75 kilogram
area,1 meter ** 2
rho,1.2 kilogram / meter ** 3
v_term,60.0 meter / second
L,25 meter
k,40.0 newton / meter


Now here's a version of `make_system` that takes a `Params` object as a parameter.

`make_system` uses the given value of `v_term` to compute the drag coefficient `C_d`.

In [4]:
def make_system(params):
    """Makes a System object for the given params.
    
    params: Params object
    
    returns: System object
    """
    unpack(params)
    
    C_d = 2 * mass * g / (rho * area * v_term**2)
    init = State(y=y_attach, v=v_init)
    t_end = 30 * s

    return System(params, C_d=C_d, 
                  init=init, t_end=t_end)

Let's make a `System`

In [5]:
system = make_system(params)
system

,values
y_attach,80 meter
v_init,0.0 meter / second
g,9.8 meter / second ** 2
mass,75 kilogram
area,1 meter ** 2
rho,1.2 kilogram / meter ** 3
v_term,60.0 meter / second
L,25 meter
k,40.0 newton / meter
C_d,0.3402777777777778 dimensionless


`spring_force` computes the force of the cord on the jumper:

In [6]:
def spring_force(y, system):
    """Computes the force of the bungee cord on the jumper:
    
    y: height of the jumper
    
    Uses these variables from system|
    y_attach: height of the attachment point
    L: resting length of the cord
    k: spring constant of the cord
    
    returns: force in N
    """
    unpack(system)
    distance_fallen = y_attach - y
    if distance_fallen <= L:
        return 0 * N
    
    extension = distance_fallen - L
    f_spring = k * extension
    return f_spring

The spring force is 0 until the cord is fully extended.  When it is extended 1 m, the spring force is 40 N. 

In [7]:
spring_force(80*m, system)

<Quantity(0, 'newton')>

In [8]:
spring_force(55*m, system)

<Quantity(0, 'newton')>

In [9]:
spring_force(54*m, system)

<Quantity(40.0, 'newton')>

`drag_force` computes drag as a function of velocity:

In [10]:
def drag_force(v, system):
    """Computes drag force in the opposite direction of `v`.
    
    v: velocity
    
    returns: drag force
    """
    unpack(system)
    f_drag = -np.sign(v) * rho * v**2 * C_d * area / 2
    return f_drag

Now here's the slope function:

In [11]:
def slope_func(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing g, rho,
            C_d, area, and mass
    
    returns: derivatives of y and v
    """
    y, v = state
    unpack(system)
    
    a_drag = drag_force(v, system) / mass
    a_spring = spring_force(y, system) / mass
    dvdt = -g + a_drag + a_spring
    
    return v, dvdt

As always, let's test the slope function with the initial params.

In [12]:
slope_func(system.init, 0, system)

(<Quantity(0.0, 'meter / second')>, <Quantity(-9.8, 'meter / second ** 2')>)

And then run the simulation.

In [13]:
ts = linspace(0, system.t_end, 301)
results, details = run_ode_solver(system, slope_func, t_eval=ts)
details

,values
sol,None
t_events,[]
nfev,236
njev,0
nlu,0
status,0
message,The solver successfully reached the end of the...
success,True


Here's the plot of position as a function of time.

In [14]:
plot_position(results)

NameError: name 'plot_position' is not defined

After reaching the lowest point, the jumper springs back almost to almost 70 m, and oscillates several times.  That looks like more osciallation that we expect from an actual jump, which suggests that there some dissipation of energy in the real world that is not captured in our model.  To improve the model, that might be a good thing to investigate.

But since we are primarily interested in the initial descent, the model might be good enough for now.

We can use `min` to find the lowest point:

In [ ]:
min(results.y)

At the lowest point, the jumper is still too high, so we'll need to increase `L` or decrease `k`.

Here's velocity as a function of time:

In [ ]:
plot_velocity(results)

In [ ]:
subplot(1, 2, 1)
plot_position(results)

subplot(1, 2, 2)
plot_velocity(results)

savefig('figs/chap09-fig03.pdf')

Although we compute acceleration inside the slope function, we don't get acceleration as a result from `run_ode_solver`.

We can approximate it by computing the numerical derivative of `ys`:

In [ ]:
a = gradient(results.v)
plot(a)
decorate(xlabel='Time (s)',
         ylabel='Acceleration (m/$s^2$)')

And we can compute the maximum acceleration the jumper experiences:

In [ ]:
max_acceleration = max(a) * m/s**2

Relative to the acceleration of gravity, the jumper "pulls" about "1.7 g's".

In [ ]:
max_acceleration / g

### Solving for length

Assuming that `k` is fixed, let's find the length `L` that makes the minimum altitude of the jumper exactly 0.

Here's the error function:

In [ ]:
def error_func(L, params):
    """Minimum height as a function of length.
    
    length: length in m
    params: Params object
    
    returns: height in m
    """
    params = Params(params, L=L)
    system = make_system(params)

    ts = linspace(0, system.t_end, 201)
    results, details = run_ode_solver(system, slope_func, t_eval=ts)
    min_height = min(results.y)
    return min_height

Let's test it with the same initial guess, `L = 100 m`:

In [ ]:
guess = 150 * m
error_func(guess, params)

And find the value of `L` we need for the world record jump:

In [ ]:
solution = fsolve(error_func, guess, params)

**Optional:** Search for the combination of length and spring constant that yields minimum height 0 while minimizing peak acceleration.

In [ ]:
# Solution

ks = np.linspace(1, 31, 11) * N / m

for k in ks:
    guess = 250 * m
    params = Params(params, k=k)
    solution = fsolve(error_func, guess, params)
    L = solution[0] * m
    params = Params(params, L=L)
    system = make_system(params)
    results, details = run_ode_solver(system, slope_func, t_eval=ts)
    a = gradient(results.v)
    g_max = max(a) * m/s**2 / g
    print(k, L, g_max)

**Optional exercise:** This model neglects the weight of the bungee cord, which might be non-negligible.  Implement the [model described here](http://iopscience.iop.org/article/10.1088/0031-9120/45/1/007) and see how different it is from our simplified model. 

### Under the hood

The gradient function in `modsim.py` adapts the NumPy function of the same name so it works with `Series` objects.


In [ ]:
%psource gradient